0) Libraries

In [59]:
import pandas as pd
import numpy as np
#import tensorflow as tf
#import keras

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

1) Download and read the data from this link:  
https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers

*Main function: pd.read_csv*


In [29]:
dataset = pd.read_csv('churn.csv')
dataset.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [30]:
dataset.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

2) Convert categorical variables (Geography,Gender) to Numerical 

*Main function: label encoding*

In [31]:
label_encoder = preprocessing.LabelEncoder()

In [32]:
for i in ['Geography', 'Gender']:
    dataset[i]= label_encoder.fit_transform(dataset[i])
dataset.loc[:5 , ['Geography', 'Gender']]

,Geography,Gender
0,0,0
1,2,0
2,0,0
3,0,0
4,2,0
5,2,1


3) Split X και y, train και test

X_Columns: 
CreditScore, Geography, Gender, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary 

Y_columns: 
Exited

and add stratify = y στα parameters από το train_test_split

*Main function: (train_test_split)*

In [33]:
X_Columns = ['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

In [34]:
Y_columns= ['Exited']

In [35]:
X = dataset.copy()[X_Columns]
y = dataset.copy()[Y_columns]
print(X.shape)
print(y.shape)

(10000, 10)
(10000, 1)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    stratify=y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 10)
(2000, 10)
(8000, 1)
(2000, 1)


4) Train with these characteristics:

* Random Forest 
* Without PCA
* Without clusters
* With Precision scoring, 
* Try with grid search 200, 300 και 500 tress, with max depth 3,5 and 10, as well as the two avaible criterion. 

*Main function: grid_search, RandomForestClassifier*

In [37]:
model_rf_class =RandomForestClassifier()

In [39]:
import pickle
file_name = 'model.pkl'
pickle.dump(model_rf_class, open(file_name, 'wb'))

In [40]:
param_grid_class = {'n_estimators': [200,300,500],
              'max_depth' : [3,5,10],
              'criterion' :['gini', 'entropy']
             }

In [41]:
grid_search_rf_class = GridSearchCV(estimator=model_rf_class, 
                              param_grid=param_grid_class, 
                              scoring= 'accuracy',
                              cv=5,
                             verbose=1)

In [42]:
grid_search_rf_class.fit(X_train,y_train.values.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [200, 300, 500]},
             scoring='accuracy', verbose=1)

In [43]:
grid_search_rf_class.best_params_

{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 300}

5) Export from the model random forest a data frame with the feature importance with the best model on each cluster. 

*Main function: feature_importance_*

In [ ]:
pd.DataFrame(data = grid_search_rf_class.best_estimator_.feature_importances_,
            index = X.columns,
            columns=['feature_importance']).sort_values(by='feature_importance',
                                                       ascending =False)
import pickle
file_name = 'model.pkl'
pickle.dump(model_rf_class, open(file_name, 'wb'))

6) Train with these characteristics:
* Algorithm Support Vector Machine
* With Standard Scaler
* With scoring Balanced Accuracy
* Try only with the kernel Radial basis function, to find gamma automatically,  while  the parameter C takes the values 0.001, 0.01, 1, 10 ,100 ,1000

*Main function: grid_search, SVC(class_weight= 'balanced'), StandarScaler, balanced_accuracy*


In [45]:
scaler = StandardScaler()

In [46]:
scaler.fit(X_train)

StandardScaler()

In [47]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [48]:
X_train_scaled = pd.DataFrame(X_train_scaled,columns = X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled,columns = X.columns) 

In [49]:
model_SVC = SVC(class_weight='balanced')

In [50]:
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': ["auto"],
                     'C': [0.001, 0.01, 1, 10, 100, 1000]}
                   ]

In [51]:
model_SVC_grid = GridSearchCV(estimator  =  model_SVC,
                              param_grid = tuned_parameters,
                              scoring="balanced_accuracy",
                              cv=5,
                              verbose = True
                             )

In [52]:
model_SVC_grid = model_SVC_grid.fit(X_train_scaled, y_train.values.ravel())  #DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel(). y = column_or_1d(y, warn=True)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [53]:
model_SVC_grid.best_params_

{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}

In [54]:
pd.DataFrame(model_SVC_grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.898882,0.027216,0.435708,0.008124,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.775332,0.746227,0.762918,0.779184,0.774046,0.767541,0.011952,1
3,1.090288,0.024508,0.428364,0.030137,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.756787,0.734422,0.744229,0.761957,0.763026,0.752084,0.011072,2
1,1.342516,0.026165,0.769713,0.042458,0.01,auto,rbf,"{'C': 0.01, 'gamma': 'auto', 'kernel': 'rbf'}",0.719395,0.715971,0.724172,0.738440,0.725670,0.724730,0.007673,3
4,2.408637,0.119305,0.428558,0.090812,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.715056,0.702822,0.716518,0.714127,0.726717,0.715048,0.007598,4
5,8.335000,0.312324,0.318953,0.027611,1000,auto,rbf,"{'C': 1000, 'gamma': 'auto', 'kernel': 'rbf'}",0.689876,0.693801,0.686344,0.685807,0.708281,0.692822,0.008244,5
0,1.670536,0.242811,0.931932,0.029285,0.001,auto,rbf,"{'C': 0.001, 'gamma': 'auto', 'kernel': 'rbf'}",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,6


In [68]:
import pickle
file_name = 'modelSVC.pkl'
pickle.dump(model_SVC_grid, open(file_name, 'wb'))

7) Predict the two models from the test sample, then export Confusion Matrix, Accuracy and Balanced_Accuracy.

*Main function: predict, confusion_matrix, accuracy, balanced_accuracy*


**Random Forest**

In [55]:
predictions_rf_class = grid_search_rf_class.predict(X_test)

In [56]:
confusion_matrix(y_test,predictions_rf_class)

array([[1560,   33],
       [ 234,  173]], dtype=int64)

In [57]:
accuracy_score(y_test,predictions_rf_class)

0.8665

In [60]:
balanced_accuracy_score(y_test,predictions_rf_class)

0.7021728970881513

**Support Vector Machine**

In [61]:
y_predictions_SVC = model_SVC_grid.predict(X_test_scaled)

In [62]:
confusion_matrix(y_test, y_predictions_SVC)

array([[1297,  296],
       [ 105,  302]], dtype=int64)

In [63]:
accuracy_score(y_test, y_predictions_SVC)

0.7995

In [64]:
balanced_accuracy_score(y_test, y_predictions_SVC)

0.7781009052195493